<a href="https://colab.research.google.com/github/TiagolGoulart/GeoPython/blob/main/PanCBERS4A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing the necessaries libraries

!pip install geopandas
!pip install rasterio
!pip install shapely
!pip install simplekml
!pip install "cbers4asat[tools] "

In [ ]:
# Importing packages

import geopandas as gpd
import simplekml
import numpy as np
import rasterio
import os
import lxml.etree as etree
import pyproj
from pyproj import Transformer
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.features import shapes
from shapely.geometry import Polygon, Point

C:\Users\tiago\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\tiago\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
C:\Users\tiago\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [ ]:
# Defining the data

# Define the coordinate system with EPSG code
sistema_coordenadas = input("Digite o código EPSG do sistema de coordenadas de referência: \n")
sistema_coordenadas = 'EPSG:'+sistema_coordenadas

# Path for mask to clip raster
kml_path = input('Cole o caminho para o kml que irá cortar as imagens raster: ')

# INPE user
user_cbers = input('Digite aqui seu e-mail de usuário cadastrado na plataforma do CBERS4A: \n')

# Inicial search date
#inicial_date = input('Digite a data inicial de busca da imagem no formato DD/MM/AAAA: \n')
d_i = int(inicial_date[:2])
m_i = int(inicial_date[3:5])
a_i = int(inicial_date[6:])

# Final search date

final_date= input('Digite a data final de busca da imagem no formato DD/MM/AAAA: \n')
d_f = int(final_date[:2])
m_f = int(final_date[3:5])
a_f = int(final_date[6:])

# Setting the results output directory

out_dir = input('Cole o caminho da pasta onde os arquivos resultantes serão salvos: \n')

In [ ]:
# Opening the extension KML and setting the bbox

# Open the KML file in binary mode
with open(kml_path, 'rb') as f:
  kml_string = f.read()

# parse the KML string into an Element object
root = etree.fromstring(kml_string)

# Get the coordinates of the polygon
polygon = root.find('.//{http://www.opengis.net/kml/2.2}Polygon')
coords_str = polygon.find('.//{http://www.opengis.net/kml/2.2}coordinates').text

coords_list_original = [tuple(map(float, coord.split(',')[0:2])) for coord in coords_str.split()]
coords_list = [(coord[1], coord[0]) for coord in coords_list_original]

# Create a Transformer for the proper UTM projection
transformer = Transformer.from_crs("EPSG:4326", sistema_coordenadas)

# Convert the coordinates to the proper UTM projection
coords_list_utm = [transformer.transform(*coord)[::-1] for coord in coords_list]
coords_list = [(coord[1], coord[0]) for coord in coords_list_utm]

# Create a shapely polygon based on UTM coordinates for cutting
polygon_utm = Polygon(coords_list)

# Defining the mask based on the initial polygon
geo = polygon_utm

# Create a shapely polygon with geographic coordinates to get bbox
polygon_bbox = Polygon(coords_list_original)

# Using the 'envelope' attribute to get a bounding rectangle and then getting the max and min values
envelope = polygon_bbox.envelope
x_min, y_min, x_max, y_max = envelope.bounds

evelope_utm = polygon_utm.envelope

In [ ]:
# Bounding box formed by [x_min, y_min, x_max, y_max]
bbox=[x_min,y_min,x_max,y_max]

from shapely.geometry import mapping
coords= [mapping(evelope_utm)]

In [ ]:
coords

[{'type': 'Polygon',
  'coordinates': (((654904.0423551123, 7451211.202710632),
    (659983.8966747862, 7451211.202710632),
    (659983.8966747862, 7455558.098957835),
    (654904.0423551123, 7455558.098957835),
    (654904.0423551123, 7451211.202710632)),)}]

In [ ]:
# Importing the libraries
from cbers4asat import Cbers4aAPI

# The date class is necessary to standardize the date format
from datetime import date

# Instantiating the object with the user registered on the platform
api = Cbers4aAPI(user_cbers)

# Bounding box formed by [x_min, y_min, x_max, y_max]
#bbox=[x_min,y_min,x_max,y_max]

# Search interval
data_inicial = date(a_i,m_i,d_i)
data_final = date(a_f,m_f,d_f)


# Querying the catalog and displaying the results
produtos = api.query(location=bbox,
                     initial_date=data_inicial,
                     end_date=data_final,
                     cloud=10,
                     limit=1,
                     collections=['CBERS4A_WPM_L4_DN'])


In [ ]:
# Viewing the data
produtos

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 'CBERS4A_WPM19814220220404',
   'collection': 'CBERS4A_WPM_L4_DN',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-43.6192, -22.4802],
      [-43.6192, -23.5585],
      [-42.4829, -23.5585],
      [-42.4829, -22.4802],
      [-43.6192, -22.4802]]]},
   'bbox': [-43.6192, -23.5585, -42.4829, -22.4802],
   'properties': {'datetime': '2022-04-04T13:27:27',
    'path': 198,
    'row': 142,
    'satellite': 'CBERS4A',
    'sensor': 'WPM',
    'cloud_cover': 10.0,
    'sync_loss': None,
    'eo:gsd': -1,
    'eo:bands': [{'name': 'pan', 'common_name': 'pan'},
     {'name': 'blue', 'common_name': 'blue'},
     {'name': 'green', 'common_name': 'green'},
     {'name': 'red', 'common_name': 'red'},
     {'name': 'nir', 'common_name': 'nir'}]},
   'assets': {'pan': {'href': 'http://www2.dgi.inpe.br/api/download/TIFF/CBERS4A/2022_04/CBERS_4A_WPM_RAW_2022_04_04.13_21_00_ETC2/198_142_0/4_BC_UTM_WGS84/CBERS_4A_WPM_2022

In [ ]:
# Downloading the images for the output directory

api.download(products=produtos,
             bands=['pan','green','blue','red','nir'],
             threads=10,
             outdir=out_dir)



In [ ]:
# Defining the path of each band image

import time
import glob
start_time = time.time()  # Get the current time before starting the loop
timeout = 120  # Timeout in seconds

while True:
    try:
        blue_path = glob.glob(out_dir + '/*BAND1.tif')[0]
        green_path = glob.glob(out_dir + '/*BAND2.tif')[0]
        red_path = glob.glob(out_dir + '/*BAND3.tif')[0]
        nir_path = glob.glob(out_dir + '/*BAND4.tif')[0]
        pan_path = glob.glob(out_dir + '/*BAND0.tif')[0]

        break  # Exit the loop if no error occurs
    except IndexError:
        if time.time() - start_time >= timeout:
            print("Timeout: The timeout limit of 120 seconds has been reached."
            + "Please wait a moment and run the cell again.")
            break  # Exit the loop if the timeout is reached

In [ ]:
out_dir_rec = out_dir+"/Recortados"

import os

# Create the directory
os.makedirs(out_dir_rec, exist_ok=True)

In [ ]:
# Saving the paths to the cutted rasters in variables

blue_path_rec=out_dir_rec+'/BAND1_REC.tif'
green_path_rec=out_dir_rec+'/BAND2_REC.tif'
red_path_rec=out_dir_rec+'/BAND3_REC.tif'
nir_path_rec=out_dir_rec+'/BAND4_REC.tif'
pan_path_rec=out_dir_rec+'/BAND0_REC.tif'

In [ ]:
# Cutting the rasters by the extension of the project with rasterio
import rasterio.mask

# Defining a cutting function of the downloaded rasters


def cut_raster(input_file,output_file):
    # Use rasterio to open the image
    with rasterio.open(input_file) as src:
    # Use the mask function to crop the image with the coordinates extracted from KML (coords)
        out_image, out_transform = rasterio.mask.mask(src, coords, crop=True)
    # Get the original raster metadata
        out_meta = src.meta
        b2=src.read(1)
        affine=src.transform
    # Update the original raster metadata with new dimensions and transformation to geographic coordinates
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})

    # Save the clipping in a new raster
    with rasterio.open(output_file, "w", **out_meta) as dest:
        dest.write(out_image)

In [ ]:
# Using the function

cut_raster(blue_path,blue_path_rec)
cut_raster(green_path,green_path_rec)
cut_raster(red_path,red_path_rec)
cut_raster(nir_path,nir_path_rec)
cut_raster(pan_path,pan_path_rec)

In [ ]:
# Create the directory
final_path = out_dir + "/FINAL_RESULT"

os.makedirs(final_path, exist_ok=True)
rgbn_path = final_path + "/rgbn_composite.tif"

# Open each band as a rasterio object
band_r = rasterio.open(red_path_rec)
band_g = rasterio.open(green_path_rec)
band_b = rasterio.open(blue_path_rec)
band_n = rasterio.open(nir_path_rec)


# Load each band's data as numpy arrays
data_r = band_r.read(1)
data_g = band_g.read(1)
data_b = band_b.read(1)
data_n = band_n.read(1)

# Get the transformation and CRS information of the reference band
transform = band_r.transform
crs = band_r.crs

# Create an RGBN matrix by stacking the bands in the correct order
rgbn_stack = np.stack([data_r, data_g, data_b, data_n], axis=0)

# Save the RGB stack to a new file
with rasterio.open(rgbn_path, 'w', driver='GTiff', width=band_r.width, height=band_r.height,
                   count=4, dtype=rgbn_stack.dtype, transform=transform, crs=crs) as dst:
    dst.write(rgbn_stack, indexes=[1, 2, 3, 4])


In [ ]:
# Close the raster objects of the bands
band_r.close()
band_g.close()
band_b.close()
band_n.close()

In [ ]:
# Pansharpening for 2m spatial resolution

from osgeo_utils.gdal_pansharpen import gdal_pansharpen

pansharpened_path = final_path + "/pansharpened.tif"

# Perforate the merge by setting the output name as well
gdal_pansharpen(['', '-b', '1','-b', '2','-b', '3','-b','4',pan_path_rec, rgbn_path, pansharpened_path])

0

In [ ]:
# Removing files to liberate memory

import os

# Home folder path
pasta = out_dir

# Get the list of files in the folder
arquivos = os.listdir(pasta)

# Go through the list of files and delete them
for arquivo in arquivos:
    caminho_arquivo = os.path.join(pasta, arquivo)
    if os.path.isfile(caminho_arquivo):
        os.remove(caminho_arquivo)


# Clipped folder path
pasta = out_dir_rec

# Get the list of files in the folder
arquivos = os.listdir(pasta)

# Go through the list of files and delete them
for arquivo in arquivos:
    caminho_arquivo = os.path.join(pasta, arquivo)
    if os.path.isfile(caminho_arquivo):
        os.remove(caminho_arquivo)